# Prerequisites
Given an integer n representing the number of courses labeled from 0 to n - 1, and an array of prerequisite pairs, determine if it's possible to enroll in all courses.

Each prerequisite is represented as a pair [a, b], indicating that course a must be taken before course b.

**Example:**
```python
Input: n = 3, prerequisites = [[0, 1], [1, 2], [2, 1]]
Output: False
```

**Constraints:**
- For any prerequisite [a, b], a will not equal b.

## Intuition

Our goal is to determine whether it is possible to enroll in all courses. Let's first identify situations where enrollment is impossible.

Consider a simple case with just two courses. If each course is a prerequisite for the other, a cycle forms in the graph representation of the courses, making enrollment impossible.

This highlights a key observation: it is **impossible to enroll in all courses if there is a circular dependency** between them. In other words, the course dependency graph **must not contain a cycle** for full enrollment to be possible.

Another important point is that the first courses that can be completed are those without prerequisites. In the graph representation, such courses have **no incoming edges**. We define the number of incoming directed edges to a node as its **in-degree**.

---

## Topological Sorting

To solve this problem, we use **topological sorting**, which orders the vertices of a graph such that for every directed edge `u -> v`, node `u` appears before node `v` in the ordering.

A well-known algorithm for performing topological sorting is **Kahn’s Algorithm**.

---

## Kahn’s Algorithm

The first step of Kahn’s Algorithm is to compute the **in-degree** of each course. This is done by counting how many times each course appears as a dependent in the prerequisite pairs.  
For a pair `[a, b]`, course `b` depends on course `a`, so the in-degree of course `b` is incremented by 1.

For example, given the prerequisites:

```python
prerequisites = [[0, 1], [0, 2], [3, 2], [1, 4], [2, 4], [4, 5]]
```

We compute the in-degrees:

```python
in_degrees = [0, 1, 2, 0, 2, 1]
```

---

Next, we process all courses with an **in-degree of 0** first. These courses have no prerequisites, so they can be taken immediately. We add them to a queue for processing:

```python
in_degrees = [0, 1, 2, 0, 2, 1] queue = [0, 3]
```

---

Now, we start processing the queue:

1. **Pop the first course from the queue:** Course `0`.
2. **Reduce the in-degree of its dependent courses:** For each course that has course `0` as a prerequisite (i.e., courses pointed to by course `0`), decrement their in-degree by 1.

After processing course `0`:

```python
in_degrees = [0, 0, 1, 0, 2, 1] queue = [3]
```

If any course's in-degree becomes `0`, add it to the queue. Course `1` now has an in-degree of `0`, so we add it:

```python
in_degrees = [0, 0, 1, 0, 2, 1] queue = [3, 1]
```


---

We repeat this process until the queue is empty. At this point:

- If we have processed **all `n` courses**, it means there is no cycle, and enrollment is **possible**.
- If we **could not process all `n` courses**, a cycle exists, making enrollment **impossible**.

In [1]:
from typing import List
from collections import deque
from collections import defaultdict

def prerequisites(n: int, prerequisites: List[List[int]]) -> bool:
    graph = defaultdict(list)
    in_degrees = [0] * n

    for prerequisite, course in prerequisites:
        graph[prerequisite].append(course)
        in_degrees[course] += 1
    
    queue = deque()
    for i in range(n):
        if in_degrees[i] == 0:
            queue.append(i)
        
    enrolled_courses = 0
    while queue:
        node = queue.popleft()
        enrolled_courses += 1

        for neighbor in graph[node]:
            in_degrees[neighbor] -= 1

            if in_degrees[neighbor] == 0:
                queue.append(neighbor)
    
    return enrolled_courses == n

## Complexity Analysis

### Time complexity
The time complexity of prerequisites is O(n + e), where e denotes the number of edges derived from the prerequisites array. Here's why:
- Creating the adjacency list and recording the in-degrees takes O(e) time because we iterate through each prerequisite once.
- Adding all courses with in-degree 0 to the queue takes O(n) time because we check the in-degree of each course once.
- Performing Kahn's algorithm takes O(n + e) time because each course and prerequisite is processed at most once during the traversal.

### Space complexity
The space complexity is O(n + e), since the adjacency list takes up O(n + e) space, while the in_degrees array and queue each take up O(n) space.
